# IV Prepare the data

This notebook assumes, that notebook 2_get_California_housing_data.ipynb has been successfully executed.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

In [2]:
def split_features_labels(filename = '../data/train_set.csv', 
                          target_col='median_house_value'):
    df = pd.read_csv(filename, index_col=0)
    X = df.drop(target_col, axis=1)
    y = df[target_col].copy()
    return X, y

housing, housing_labels = split_features_labels()
housing_test, housing_test_labels = split_features_labels('../data/test_set.csv')

## IV.1 Data Cleaning

In [3]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 1862 to 14525
Data columns (total 9 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16344 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
ocean_proximity       16512 non-null object
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [4]:
# Identify features with missing values
housing.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        168
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [5]:
mask = housing.total_bedrooms.isna()
housing[mask].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
5059,-118.35,34.01,35.0,3776.0,NaN,1583.0,749.0,3.5486,<1H OCEAN
14015,-117.17,32.75,52.0,1052.0,NaN,381.0,201.0,3.0726,NEAR OCEAN
11311,-117.96,33.78,33.0,1520.0,NaN,658.0,242.0,4.8750,<1H OCEAN
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
14173,-117.05,32.75,36.0,2024.0,NaN,1030.0,390.0,3.8233,NEAR OCEAN


In [6]:
housing.shape

(16512, 9)

In [7]:
# variant 1: Drop rows with missing values
h1 = housing.dropna(subset = ['total_bedrooms'])
h1.shape

(16344, 9)

In [8]:
# variant 2: Drop columns with missing values
h2 = housing.drop('total_bedrooms', axis=1)
h2.shape

(16512, 8)

In [9]:
# variant 3: Impute missing values
median = housing["total_bedrooms"].median()
h3 = housing.copy()
h3['total_bedrooms'] = housing["total_bedrooms"].fillna(median)
h3.shape

(16512, 9)

In [10]:
h3.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
ocean_proximity       0
dtype: int64

In [11]:
housing_copy = housing.copy().iloc[21:24]
median = housing_copy["total_bedrooms"].median()
housing_copy["total_bedrooms"].fillna(median, inplace=True) # option 3
housing_copy

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
2930,-119.00,35.36,39.0,896.0,217.0,805.0,197.0,1.2500,INLAND
16225,-121.33,37.97,38.0,3166.0,575.0,1351.0,561.0,3.5404,INLAND
5115,-118.30,33.96,39.0,2802.0,618.0,1524.0,529.0,2.6518,<1H OCEAN


In [12]:
h3.loc[5059]

longitude               -118.35
latitude                  34.01
housing_median_age           35
total_rooms                3776
total_bedrooms              435
population                 1583
households                  749
median_income            3.5486
ocean_proximity       <1H OCEAN
Name: 5059, dtype: object

## Impute numerical features with scikit-learn Transformer

In [13]:
# Create DataFrame with numerical features
housing_num = housing.drop("ocean_proximity", axis=1)
housing_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1862,-124.15,41.81,17.0,3276.0,628.0,3546.0,585.0,2.2868
18832,-122.92,41.70,23.0,4017.0,792.0,1634.0,619.0,2.3571
2488,-120.35,36.14,9.0,2671.0,647.0,1484.0,541.0,1.7075
7637,-118.29,33.82,21.0,4383.0,901.0,2689.0,913.0,3.4375
6941,-118.10,33.99,31.0,965.0,217.0,599.0,206.0,2.7202


In [14]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy='median')
imputer.fit(housing_num)
X = imputer.transform(housing_num)
type(X)

numpy.ndarray

In [15]:
# Create DataFrame from imputed feature matrix
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                         index=housing_num.index)
housing_tr.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [16]:
housing_tr.loc[5059]

longitude             -118.3500
latitude                34.0100
housing_median_age      35.0000
total_rooms           3776.0000
total_bedrooms         435.0000
population            1583.0000
households             749.0000
median_income            3.5486
Name: 5059, dtype: float64

In [17]:
imputer.statistics_

array([ -118.49  ,    34.25  ,    29.    ,  2127.    ,   435.    ,
        1167.    ,   410.    ,     3.5348])

In [18]:
housing_num.median().values

array([ -118.49  ,    34.25  ,    29.    ,  2127.    ,   435.    ,
        1167.    ,   410.    ,     3.5348])

In [19]:
imputer.strategy

'median'

In [20]:
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1862,-124.15,41.81,17.0,3276.0,628.0,3546.0,585.0,2.2868
18832,-122.92,41.70,23.0,4017.0,792.0,1634.0,619.0,2.3571
2488,-120.35,36.14,9.0,2671.0,647.0,1484.0,541.0,1.7075
7637,-118.29,33.82,21.0,4383.0,901.0,2689.0,913.0,3.4375
6941,-118.10,33.99,31.0,965.0,217.0,599.0,206.0,2.7202


## Encode categorical variable

In [21]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded

array([4, 1, 1, ..., 0, 4, 0])

In [22]:
print(encoder.classes_)

['<1H OCEAN' 'INLAND' 'ISLAND' 'NEAR BAY' 'NEAR OCEAN']


In [23]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [24]:
housing_cat_1hot.toarray()

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.]])

In [25]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
encoder.fit_transform(housing_cat)

array([[0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ..., 
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [26]:
encoder.classes_

array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype='<U10')

In [27]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

In [28]:
LabelBinarizer().fit_transform(DataFrameSelector(cat_attribs).fit_transform(housing))

array([[0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ..., 
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [29]:
class Binarizer(BaseEstimator, TransformerMixin):
    "Work around until CategoricalEncoder is in stable branch"
    def __init__(self):
        self.binarizer = LabelBinarizer()
    def fit(self, X, y=None):
        return self.binarizer.fit(X)
    def transform(self, X, y=None):
        return self.binarizer.transform(X)
    
Binarizer().fit_transform(DataFrameSelector(cat_attribs).fit_transform(housing))

array([[0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ..., 
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [30]:
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.added_columns = ["rooms_per_household", "population_per_household"]
        if self.add_bedrooms_per_room:
            self.added_columns += ["bedrooms_per_room"]
            
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

housing_extra_attribs = pd.DataFrame(housing_extra_attribs, 
                                     columns=list(housing.columns) + attr_adder.added_columns)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
0,-124.15,41.81,17,3276,628,3546,585,2.2868,NEAR OCEAN,5.6,6.06154
1,-122.92,41.7,23,4017,792,1634,619,2.3571,INLAND,6.4895,2.63974
2,-120.35,36.14,9,2671,647,1484,541,1.7075,INLAND,4.93715,2.74307
3,-118.29,33.82,21,4383,901,2689,913,3.4375,<1H OCEAN,4.80066,2.94524
4,-118.1,33.99,31,965,217,599,206,2.7202,<1H OCEAN,4.68447,2.90777


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

num_pipeline.fit_transform(housing_num)

array([[-2.2813647 ,  2.88761852, -0.92730526, ...,  0.06049651,
         0.25526898, -0.34686806],
       [-1.66841867,  2.83618177, -0.45044749, ...,  0.40200195,
        -0.03977906, -0.26067972],
       [-0.3877103 ,  0.23628785, -1.56311561, ..., -0.19399019,
        -0.0308696 ,  0.45012399],
       ..., 
       [ 0.47938409, -0.69424972,  0.26483915, ...,  0.21253074,
        -0.07214738, -0.50810062],
       [-1.71825168,  1.32113568,  1.85436503, ...,  1.10348168,
        -0.09294394,  0.28343112],
       [ 1.21691264, -1.26940611, -1.00678155, ...,  0.0333381 ,
         0.05293733, -0.47430168]])

In [32]:
from sklearn.pipeline import FeatureUnion

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])


# Monkey-patch function returning feature names
all_num_columns = num_attribs + num_pipeline.named_steps['attribs_adder'].added_columns
num_pipeline.get_feature_names = lambda: all_num_columns

# Modify class by inheritance
cat_pipeline_steps = [
        ('selector', DataFrameSelector(cat_attribs)),
        ('label_encoder', Binarizer()),
    ]
class MyPipeline(Pipeline):
    def get_feature_names(self):
        return self.steps[1][1].binarizer.classes_
cat_pipeline = MyPipeline(cat_pipeline_steps)

preparation_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [33]:
cat_pipeline.fit_transform(housing)

array([[0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ..., 
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [34]:
obj = cat_pipeline.steps[1][1]
obj.binarizer.classes_

array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype='<U10')

In [35]:
preparation_pipeline.get_feature_names()

['num_pipeline__longitude',
 'num_pipeline__latitude',
 'num_pipeline__housing_median_age',
 'num_pipeline__total_rooms',
 'num_pipeline__total_bedrooms',
 'num_pipeline__population',
 'num_pipeline__households',
 'num_pipeline__median_income',
 'num_pipeline__rooms_per_household',
 'num_pipeline__population_per_household',
 'num_pipeline__bedrooms_per_room',
 'cat_pipeline__<1H OCEAN',
 'cat_pipeline__INLAND',
 'cat_pipeline__ISLAND',
 'cat_pipeline__NEAR BAY',
 'cat_pipeline__NEAR OCEAN']

In [36]:
X_train = pd.DataFrame(preparation_pipeline.fit_transform(housing),
                      columns=preparation_pipeline.get_feature_names(),
                      index = housing.index)
X_train.head()

,num_pipeline__longitude,num_pipeline__latitude,num_pipeline__housing_median_age,num_pipeline__total_rooms,num_pipeline__total_bedrooms,num_pipeline__population,num_pipeline__households,num_pipeline__median_income,num_pipeline__rooms_per_household,num_pipeline__population_per_household,num_pipeline__bedrooms_per_room,cat_pipeline__<1H OCEAN,cat_pipeline__INLAND,cat_pipeline__ISLAND,cat_pipeline__NEAR BAY,cat_pipeline__NEAR OCEAN
1862,-2.281365,2.887619,-0.927305,0.292908,0.217002,1.856885,0.224303,-0.830593,0.060497,0.255269,-0.346868,0.0,0.0,0.0,0.0,1.0
18832,-1.668419,2.836182,-0.450447,0.633004,0.608756,0.181753,0.313571,-0.793651,0.402002,-0.039779,-0.260680,0.0,1.0,0.0,0.0,0.0
2488,-0.387710,0.236288,-1.563116,0.015231,0.262388,0.050336,0.108780,-1.135007,-0.193990,-0.030870,0.450124,0.0,1.0,0.0,0.0,0.0
7637,0.638850,-0.848560,-0.609400,0.800987,0.869129,1.106054,1.085476,-0.225916,-0.246395,-0.013438,-0.128123,1.0,0.0,0.0,0.0,0.0
6941,0.733532,-0.769067,0.185363,-0.767771,-0.764773,-0.725026,-0.770771,-0.602847,-0.291004,-0.016668,0.176319,1.0,0.0,0.0,0.0,0.0


In [37]:
import os
result_path = '../results'
if not os.path.exists(result_path):
    os.makedirs(result_path)
get_path = lambda fn: os.path.join(result_path, fn)
get_path('X_train.csv')

'../results/X_train.csv'

In [38]:
X_train.to_csv(get_path('X_train.csv'))
housing_labels.to_csv(get_path('y_train.csv'), header=True)

## Prepare test data

In [39]:
X_test = pd.DataFrame(preparation_pipeline.transform(housing_test),
                      columns=preparation_pipeline.get_feature_names(),
                     index = housing_test.index)
X_test.to_csv(get_path('X_test.csv'))
housing_test_labels.to_csv(get_path('y_test.csv'), header=True)